Data from here: https://mengtingwan.github.io/data/goodreads.html#datasets


In [ ]:
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd
from itertools import islice
import psycopg2
from psycopg2.extras import execute_values
pd.options.display.float_format = '{:,}'.format

def batched(iterable, n):
    "Batch data into tuples of length n. The last batch may be shorter."
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while (batch := tuple(islice(it, n))):
        yield batch

# convert string to int or none
def parseInt(string):
    return int(float(string)) if string else None

In [ ]:
# setup db

con = psycopg2.connect(
    host="localhost",
    database="reviews",
    user="postgres",
    password="postgres"
    )
cur = con.cursor()

cur.execute('''
            CREATE TABLE IF NOT EXISTS work(
                id INTEGER PRIMARY KEY,
                title TEXT NOT NULL,
                year INTEGER
            )''')
cur.execute('''
            CREATE TABLE IF NOT EXISTS author(
                id INTEGER PRIMARY KEY,
                name TEXT NOT NULL
            )''')
cur.execute('''
            CREATE TABLE IF NOT EXISTS book(
                id INTEGER PRIMARY KEY,
                workId INTEGER NOT NULL,
                title TEXT NOT NULL,
                titleWithoutSeries TEXT NOT NULL,
                url TEXT NOT NULL, 
                year INTEGER,
                rating REAL,
                bestOfWork BOOLEAN NOT NULL,
                FOREIGN KEY (workId)
                    REFERENCES work (id)
                    ON UPDATE CASCADE ON DELETE CASCADE
            )''')
cur.execute('''
            CREATE TABLE IF NOT EXISTS authorForBook(
                bookId INTEGER NOT NULL,
                authorId INTEGER NOT NULL,
                role TEXT,
                PRIMARY KEY (bookId, authorId),
                FOREIGN KEY (bookId)
                    REFERENCES book(id)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (authorId)
                    REFERENCES author (id)
                    ON UPDATE CASCADE ON DELETE CASCADE
            )
            ''')
cur.execute('''
            CREATE TABLE IF NOT EXISTS review(
                id TEXT PRIMARY KEY, 
                bookId INTEGER NOT NULL,
                rating INTEGER, 
                text TEXT NOT NULL, 
                date TEXT NOT NULL, 
                nVotes INTEGER NOT NULL,
                FOREIGN KEY (bookId) 
                    REFERENCES book (id)
                    ON UPDATE CASCADE ON DELETE CASCADE
            )''')
con.commit()

# Save data into db

In [ ]:
# works
bestBooksForWorks = {}
file_name = 'goodreads_book_works.json.gz'
with gzip.open(file_name) as works:
    data = []
    for work in works:
        d = json.loads(work)
        data.append(
            (
                int(d['work_id']), 
                d['original_title'],
                parseInt(d['original_publication_year'])
                )
        )
        bestBooksForWorks[int(d['work_id'])] = int(d['best_book_id'])
    execute_values(cur, 'insert into work values %s', data)
    con.commit()

In [ ]:
bestBooksForWorks = {}
file_name = 'goodreads_book_works.json.gz'
with gzip.open(file_name) as works:
    data = []
    for work in works:
        d = json.loads(work)
        if d['original_title'] == '':
            print(d)

In [ ]:
# authors
file_name = 'goodreads_book_authors.json.gz'
with gzip.open(file_name) as authors:
    data = [
        (
            d['author_id'], 
            d['name'],
            )
        for author in authors
        if (d := json.loads(author))
    ]
    execute_values(cur, 'insert into author values %s', data)
    con.commit()

In [ ]:
# books
file_name = 'goodreads_books.json.gz'
with gzip.open(file_name) as books:
    data = []
    author_data = []
    for book in books:
        d = json.loads(book)
        if d['work_id'] == '':
            continue
        data.append(
            (
                int(d['book_id']),
                int(d['work_id']),
                d['title'], 
                d['title_without_series'],
                d['link'], 
                parseInt(d['publication_year']), 
                parseInt(d['average_rating']),
                bestBooksForWorks[int(d['work_id'])] == int(d['book_id'])
            )
        )
        for author in d['authors']:
            author_data.append(
                (
                    int(d['book_id']),
                    int(author['author_id']),
                    author['role']
                )
            )
    execute_values(cur, 'insert into book values %s', data)
    execute_values(cur, 'insert into authorForBook values %s on conflict do nothing', author_data)
    con.commit()


In [ ]:
# reviews
file_name = 'goodreads_reviews_dedup.json.gz'
with gzip.open(file_name) as reviews:
    for i, review_batch in enumerate(batched(reviews, 1_000_000)):
        data = []
        for review in review_batch:
            d = json.loads(review)
            cur.execute(f'select * from book where id = {int(d["book_id"])}')
            if cur.fetchone() is not None:
                data.append(
                    (
                        d['review_id'],
                        int(d['book_id']),
                        parseInt(d['rating']),
                        d['review_text'],
                        d['date_added'],
                        int(d['n_votes'])
                    )
                )
        execute_values(cur, 'insert into review values %s', data)

        # trying to avoid errors on reviews without a book in db without querying everytime but didn't work 
        # execute_values(cur, '''
        #     insert into review(id, bookId, rating, text, date, nVotes) 
        #     select val.id, val.bookId, val.rating, val.text, val.date, val.nVotes
        #     from (
        #         values %s
        #     ) val (id, bookId, rating, text, date, nVotes)
        #     join book on book.id = val.bookId
        #     ''', data)
        con.commit()
        print(f'{i+1} batches processed')
        

# Data Overview

In [ ]:
# do sum instead of 'count(book.id)' as count() will count every row, even nulls meaning every book would have at least 1 review
cur.execute('''
            select book.id, title, sum((case when review.bookId is not null then 1 else 0 end))
            from book 
            left join review on book.id = review.bookId 
            group by book.id 
            order by sum((case when review.bookId is not null then 1 else 0 end)) desc
            ''')
df = pd.DataFrame(cur.fetchall(), columns=['id', 'title', 'count'])

In [ ]:
df['count'].plot(kind='hist', logy=True, bins=100);

In [ ]:
df[df['count'] < 1000]['count'].plot(kind='hist', logy=True, bins=100);

In [ ]:
df[df['count'] < 100]['count'].plot(kind='hist', logy=True, bins=100);

In [ ]:
for n in [10, 20, 50, 100, 200]:
    print(f'Taking only books with at least {n} reviews:')
    print(f'{len(df[df["count"] >= n])} books with {df[df["count"] >= n]["count"].sum()} total reviews\n')
    

# Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')


In [ ]:
cur.execute('''
            CREATE TABLE model (
                id SERIAL PRIMARY KEY,
                name TEXT NOT NULL,
            )
''')
cur.execute('''
        create table review_embed (
            id INTEGER PRIMARY KEY,
            modelId INTEGER NOT NULL,
            embedding vector(384) NOT NULL,
            FOREIGN KEY (id)
                REFERENCES review (id)
                ON UPDATE CASCADE ON DELETE CASCADE
            FOREIGN KEY (modelId)
                REFERENCES model (id)
                ON UPDATE CASCADE ON DELETE CASCADE
        );
''')
con.commit()

In [ ]:
"""
    'The Two Towers (The Lord of the Rings, #2)',
    'Pride and Prejudice',
    'And Then There Were None', 
    'Guards! Guards! (Discworld, #8)', 
    'Foundation (Foundation, #1)', 
    'The ABC Murders',
    'Frankenstein'
"""
for book_title in [
    'The Dark Forest (Remembrance of Earth\'\'s Past, #2)',
    ]:
    cur.execute('''
                select review.id, review.text
                from review
                left join book on review.bookId = book.id
                where book.title like '%s'
                group by review.id, book.id
    ''' % book_title)
    df = pd.DataFrame(cur.fetchall(), columns=['id', 'text'])
    print(f'Computing embeddings for {book_title}')
    data = [
        (
            review['id'],
            model.encode(review['text']).tolist()
        )
        for _, review in df.iterrows()
    ]

    execute_values(cur, 'insert into review_embed values %s', data)
    con.commit()
    

In [ ]:
query_embedding = model.encode('fun and exciting mystery')
cur.execute('''
            select review.text, book.title
            from review_embed
            inner join review on review_embed.id = review.id
            inner join book on review.bookId = book.id
            order by embedding <=> '%s'
            limit 15
''' % query_embedding.tolist())

df = pd.DataFrame(cur.fetchall(), columns=['text', 'title'])

In [ ]:
df